### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Written Description of Observable Trends
* The first trend I noticed was in the school summary table that we created. Percent of students passing overall seems to be correlated to the size of the school. All schools with student populations of less than 2000 students had % overall passing greater than 85%. This could likely be due to smaller class sized and a larger teacher to student ratio.
* Secondly, when grouped by school, average math and reading scores are similar across grades. There are no drastic changes between 9th-12th grades.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [11]:
#make arrays of different data
school_names = school_data['school_name'].unique()

school_count = len(school_names)

district_students = school_data['size'].sum()

total_student = student_data['school_name'].count()

total_budget = school_data['budget'].sum()

#calculate passing grades for reading & math
num_pass_read = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
percent_pass_read = num_pass_read/total_student

num_pass_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
percent_pass_math = num_pass_math/total_student

#calculate average scores
avg_read_score = student_data['reading_score'].mean()
avg_math_score = student_data['math_score'].mean()

#calculate overall % passing
overall_pass = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]['school_name'].count()/total_student

#create data frame

district_summary = pd.DataFrame({
    "Total Schools": [school_count], 
    "Total Students": [district_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_read_score],
    "% Passing Math": [percent_pass_math],
    "% Passing Reading": [percent_pass_read],
    "% Overall Passing": [overall_pass]
})

#change order

dist_sum = district_summary[["Total Schools", "Total Students", 
                            "Total Budget", "Average Math Score", 
                            "Average Reading Score", "% Passing Math",
                            "% Passing Reading", "% Overall Passing"]]

dist_sum.style.format({"Total Budget": "${:,.2f}",
                      "Average Reading Score": "{:.1f}",
                      "Average Math Score": "{:.1f}",
                      "% Passing Math": "{:.1%}", 
                      "% Passing Reading": "{:.1%}",
                      "Overall Passing Rate": "{:.1%}"})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",79.0,81.9,75.0%,85.8%,0.651723


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [32]:
#groups by schools
by_school = school_data_complete.set_index('school_name').groupby(['school_name'])

#school type
school_types = school_data.set_index('school_name')['type']

#total students
stu_per_school = by_school['Student ID'].count()

#total budget for school & student
school_budget = school_data.set_index('school_name')['budget']
student_budget = school_data.set_index('school_name')['budget']

#average math and reading scores
avg_math = by_school['math_score'].mean()
avg_reading = by_school['reading_score'].mean()

# % passing math and reading, overall passing %
passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_school
passing_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_school
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_school

#make new dataframe
school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": stu_per_school,
    "Total School Budget": school_budget,
    "Per Student Budget": student_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_reading,
    "% Passing Math": passing_math,
    "% Passing Reading": passing_read,
    "% Overall Passing": overall
})

school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget',
                                'Per Student Budget', 'Average Math Score',
                                'Average Reading Score', '% Passing Math', '% Passing Reading',
                                '% Overall Passing']]

school_summary.style.format({'Total Students': '{:,}',
                            'Total School Budget': '${:,}',
                            'Per Student Budget': '${:.0f}',
                            'Average Math Score': '{:.1f}',
                            '% Passing Math': '{:.1%}',
                            '% Passing Reading': '{:.1%}',
                            '% Overall Passing': '{:.1%}'})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$3124928,77.0,81.033963,66.7%,81.9%,54.6%
Cabrera High School,Charter,"1,858","$1,081,356",$1081356,83.1,83.975780,94.1%,97.0%,91.3%
Figueroa High School,District,"2,949","$1,884,411",$1884411,76.7,81.158020,66.0%,80.7%,53.2%
Ford High School,District,"2,739","$1,763,916",$1763916,77.1,80.746258,68.3%,79.3%,54.3%
Griffin High School,Charter,"1,468","$917,500",$917500,83.4,83.816757,93.4%,97.1%,90.6%
Hernandez High School,District,"4,635","$3,022,020",$3022020,77.3,80.934412,66.8%,80.9%,53.5%
Holden High School,Charter,427,"$248,087",$248087,83.8,83.814988,92.5%,96.3%,89.2%
Huang High School,District,"2,917","$1,910,635",$1910635,76.6,81.182722,65.7%,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$3094650,77.1,80.966394,66.1%,81.2%,53.5%
Pena High School,Charter,962,"$585,858",$585858,83.8,84.044699,94.6%,95.9%,90.5%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [26]:
top_5_schools = school_summary.sort_values("% Overall Passing", ascending = False)
top_5_schools.head().style.format({'Total Students': '{:,}',
                            'Total School Budget': '${:,}',
                            'Per Student Budget': '${:.0f}',
                            'Average Math Score': '{:.2f}',
                            '% Passing Math': '{:.2%}',
                            '% Passing Reading': '{:.2%}',
                            '% Overall Passing': '{:.2%}'})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$1081356,83.06,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$1043130,83.42,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$917500,83.35,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$1319574,83.27,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$585858,83.84,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [27]:
bottom_5_schools = school_summary.sort_values("% Overall Passing", ascending = True)
bottom_5_schools.head().style.format({'Total Students': '{:,}',
                            'Total School Budget': '${:,}',
                            'Per Student Budget': '${:.0f}',
                            'Average Math Score': '{:.2f}',
                            '% Passing Math': '{:.2%}',
                            '% Passing Reading': '{:.2%}',
                            '% Overall Passing': '{:.2%}'})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$2547363,76.84,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$1884411,76.71,81.158020,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$1910635,76.63,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$3022020,77.29,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$3094650,77.07,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [31]:
math_ninth = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
math_tenth = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
math_eleventh = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
math_twelfth = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

math_scores = pd.DataFrame({"9th": math_ninth, "10th": math_tenth,
                           "11th": math_eleventh, "12th":math_twelfth})

math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

math_scores.style.format({'9th': '{:.6f}', '10th': '{:.6f}', '11th': '{:.6f}', '12th': '{:.6f}'})



,9th,10th,11th,12th
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [33]:
read_ninth = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
read_tenth = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
read_eleventh = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
read_twelfth = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

reading_scores = pd.DataFrame({"9th": read_ninth, "10th": read_tenth,
                           "11th": read_eleventh, "12th": read_twelfth})

reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

reading_scores.style.format({'9th': '{:.6f}', '10th': '{:.6f}', '11th': '{:.6f}', '12th': '{:.6f}'})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [37]:
bins = [0, 584.999, 628.999, 643.999, 674.999]
group_name = ['<$584', '$585-629', '$630-644', '$645-675']
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins, labels = group_name)

by_spending = school_data_complete.groupby('spending_bins')

avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >=70) & (school_data_complete['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()

spend_score = pd.DataFrame({'Average Math Score': avg_math,
                           'Average Reading Score': avg_read,
                           '% Passing Math': pass_math,
                           '% Passing Reading': pass_read,
                           '% Overall Passing': overall})

spend_score = spend_score[{'Average Math Score', 'Average Reading Score',
                          '% Passing Math', '% Passing Reading', '% Overall Passing'}]

spend_score.index.name = "Spending Ranges (Per Student)"
spend_score = spend_score.reindex(group_name)

spend_score.style.format({'Average Math Score': '{:.2f}',
                         'Average Reading Score': '{:.2f}',
                         '% Passing Math': '{:.2f}',
                         '% Passing Reading': '{:.2f}',
                         '% Overall Passing': '{:.2f}'})



,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing,Average Math Score
Spending Ranges (Per Student),,,,,
<$584,83.96,0.97,0.94,0.91,83.36
$585-629,82.31,0.89,0.79,0.71,79.98
$630-644,81.48,0.84,0.71,0.60,78.05
$645-675,80.96,0.81,0.67,0.54,77.06


## Scores by School Size

* Perform the same operations as above, based on school size.

In [43]:
bins = [0, 999.999, 1999.999, 4999.999]
group_name = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], bins, labels = group_name)

by_size = school_data_complete.groupby('size_bins')

avg_math = by_size['math_score'].mean()
avg_read = by_size['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >=70) & (school_data_complete['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()

size_score = pd.DataFrame({'Average Math Score': avg_math,
                           'Average Reading Score': avg_read,
                           '% Passing Math': pass_math,
                           '% Passing Reading': pass_read,
                           '% Overall Passing': overall})

size_score = size_score[{'Average Math Score', 'Average Reading Score',
                          '% Passing Math', '% Passing Reading', '% Overall Passing'}]

size_score.index.name = "School Size"
size_score = size_score.reindex(group_name)

size_score.style.format({'Average Math Score': '{:.2f}',
                         'Average Reading Score': '{:.2f}',
                         '% Passing Math': '{:.2}',
                         '% Passing Reading': '{:.2}',
                         '% Overall Passing': '{:.2}'})

,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing,Average Math Score
School Size,,,,,
Small (<1000),83.97,0.96,0.94,0.9,83.83
Medium (1000-2000),83.87,0.97,0.94,0.91,83.37
Large (2000-5000),81.20,0.82,0.69,0.57,77.48


## Scores by School Type

* Perform the same operations as above, based on school type

In [46]:
by_type= school_data_complete.groupby('type')

avg_math = by_type['math_score'].mean()
avg_read = by_type['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >=70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

type_score = pd.DataFrame({'Average Math Score': avg_math,
                           'Average Reading Score': avg_read,
                           '% Passing Math': pass_math,
                           '% Passing Reading': pass_read,
                           '% Overall Passing': overall})

type_score = type_score[{'Average Math Score', 'Average Reading Score',
                          '% Passing Math', '% Passing Reading', '% Overall Passing'}]

type_score.index.name = "Type of School"

type_score.style.format({'Average Math Score': '{:.2f}',
                         'Average Reading Score': '{:.2f}',
                         '% Passing Math': '{:.2f}',
                         '% Passing Reading': '{:.2f}',
                         '% Overall Passing': '{:.2f}'})

,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing,Average Math Score
Type of School,,,,,
Charter,83.90,0.97,0.94,0.91,83.41
District,80.96,0.81,0.67,0.54,76.99
